<a href="https://colab.research.google.com/github/andrewjang74/TensorFlow-Tutorials/blob/master/%5BAI_Essential%5D_Document_Classification_%EC%86%94%EB%A3%A8%EC%85%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 로이터 뉴스 분류
- **목표**
  - 이 워크샵의 목표는 로이터 뉴스 기사 데이터셋을 사용하여 딥러닝 모델을 학습시키고, 다양한 카테고리의 뉴스 기사를 분류하는 것입니다.

- **데이터셋 정보**
  - 데이터는 (batch, sequence_length) 형태로, 여기서 batch는 한 번에 처리하는 뉴스 기사 수, sequence_length는 각 기사의 단어 시퀀스 길이를 나타냅니다.
  - 데이터셋은 총 46개의 클래스(예: 무역, 스포츠, 경제, 기술, 의료 등)로 구성되어 있습니다.
  - 데이터는 훈련용 데이터와 테스트용 데이터로 나뉘며, 훈련 데이터는 모델 학습에 사용되고, 테스트 데이터는 학습한 모델의 성능을 평가하는 데 사용됩니다.

- **문제 유형**
  - 이 워크샵은 다중 클래스 분류 문제로, 각 뉴스 기사가 어떤 카테고리에 속하는지를 예측하는 것이 목표입니다. 모델의 성능은 Accuracy 지표로 측정됩니다.

## 1. 환경 설정

In [ ]:
%%capture
!pip install JAEN -qU

In [ ]:
# 그대로 실행하세요.
import torch
import torch.nn as nn
import torch.optim as optim
from torchinfo import summary

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
# 사용자명을 입력하세요. (이름이 아니여도 괜찮습니다.)
username = "<sol>"
assert username, "username 변수에 값이 설정되지 않았습니다."

In [ ]:
# 그대로 실행하세요.
from JAEN.competition import Competition
comp = Competition(
    username=username,
    course_name='AI Essential',
    course_round='0224(4)',
    competition_name='Document Classification'
)

## 2. 데이터 로드

In [ ]:
from JAEN.datasets import load_documents
X, y, TEST = load_documents()
X.shape, y.shape, TEST.shape

(torch.Size([460, 100]), torch.Size([460]), torch.Size([212, 100]))

In [ ]:
import numpy as np
len(np.unique(y)), np.unique(y)

(46,
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45]))

In [ ]:
X[:5, :10]

tensor([[  547,   216, 16134,  1660,     7,     4,   993,     5,   988, 11043],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [    5,  1458,  1256,  6539,     9,   219,  1528,    87,    13,  1528],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [   22,   628,   384,    10,   176,    13,     4,   141,   740,  2687]])

## 3. 제출 예시 코드

In [ ]:
# TEST의 예측값 대입 (지금은 0으로 채워진 값 대입)
comp.prediction =  torch.zeros(212)
comp.prediction

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
# 제출
comp.submit()

[Document Classification 평가 결과]
 AI Essential 1107(1) 과정 Hello World님의 점수는 0.02358490566037736 입니다.

## 4. 순환신경망 모델을 구성하고 학습하여 TEST를 예측해보세요.
- TEST의 예측 결과는 `comp.prediction`에 대입해주세요. **torch.tensor** 형태여야합니다.

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
# TensorDataset을 사용하여 데이터셋 생성
train_dataset = TensorDataset(X, y)
test_dataset = TensorDataset(TEST)

# DataLoader 생성
batch_size = 32  # 배치 크기 설정
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
import numpy as np

for x, y in train_loader:
  print(x.shape, y.shape)
  print(x[:5, :10], y[:5])
  break

torch.Size([32, 100]) torch.Size([32])
tensor([[  47,   18,  470, 2023,   27, 1504, 1505,   31, 1273,   52],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
        [2400,  260,  489,  996,  928,  401,  401,  162, 1661, 1527],
        [  84,   27, 1768,    6,  344,  131, 2519,    4, 2167, 3030],
        [1518, 1415,   28, 1734,   72,    7,  286,  177,  237,  173]]) tensor([13, 38,  1, 34, 30])


In [ ]:
import torch
import torch.nn as nn

class ReutersLSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_size):
        super().__init__()
        # 임베딩 레이어
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        # LSTM 레이어
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        # FC 레이어
        self.fc = nn.Linear(hidden_dim, output_size)
        # Softmax는 CrossEntropyLoss에서 내부적으로 적용되므로 여기서는 생략

    def forward(self, x):
        # 입력 시퀀스를 임베딩
        x = self.embedding(x)
        # LSTM 레이어
        lstm_out, _ = self.lstm(x)
        # 마지막 시퀀스의 출력(hidden state)을 사용
        out = self.fc(lstm_out[:, -1, :])
        return out

# 모델 초기화
vocab_size = 50000  # 단어 사전 크기
embedding_dim = 128  # 임베딩 차원
hidden_dim = 256  # LSTM의 은닉 상태 크기
output_size = 46  # 로이터 뉴스의 클래스 수

model = ReutersLSTMModel(vocab_size, embedding_dim, hidden_dim, output_size)

# 모델 구조 확인
summary(model)

Layer (type:depth-idx)                   Param #
ReutersLSTMModel                         --
├─Embedding: 1-1                         6,400,000
├─LSTM: 1-2                              395,264
├─Linear: 1-3                            11,822
Total params: 6,807,086
Trainable params: 6,807,086
Non-trainable params: 0

In [ ]:
# 손실함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()  # 크로스 엔트로피 손실 함수
optimizer = optim.Adam(model.parameters())

In [ ]:
# 모델 학습 과정 구현
epochs = 100
for epoch in range(epochs):
    running_loss = 0.0
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    if epoch % 10 == 0:
       print(f'Epoch [{epoch}/{epochs}], Loss: {running_loss / len(train_loader)}')

Epoch [0/100], Loss: 3.8606094837188722
Epoch [10/100], Loss: 0.0766895482937495
Epoch [20/100], Loss: 0.012652608721206585
Epoch [30/100], Loss: 0.0071266596826414265
Epoch [40/100], Loss: 0.005854664091020823
Epoch [50/100], Loss: 0.00477049780699114
Epoch [60/100], Loss: 0.004697613829436401
Epoch [70/100], Loss: 0.004398201697040349
Epoch [80/100], Loss: 0.00396974435619389
Epoch [90/100], Loss: 0.18926401883363725


In [ ]:
# 학습된 모델의 TEST 예측
import torch.nn.functional as F

# TEST 데이터에 대한 예측
model.eval()  # 모델을 평가 모드로 설정
all_predictions = []

# DataLoader를 사용하여 배치 단위로 예측
with torch.no_grad():
    for inputs in test_loader:
        outputs = model(inputs[0])  # 배치의 입력을 모델에 전달

        # 소프트맥스를 사용하여 확률로 변환
        probabilities = F.softmax(outputs, dim=1)

        # 가장 높은 확률을 가진 클래스를 선택
        predicted_labels = torch.argmax(probabilities, dim=1)
        all_predictions.append(predicted_labels)

# 모든 배치의 예측 결과를 하나의 텐서로 결합
all_predictions = torch.cat(all_predictions)

# 예측 레이블 확인
print(all_predictions[:10])

tensor([44, 33, 29, 44, 35,  3, 44, 33, 19, 35])


In [ ]:
# comp.prediction에 TEST 예측 결과 대입
comp.prediction = all_predictions
comp.prediction

tensor([44, 33, 29, 44, 35,  3, 44, 33, 19, 35, 10, 29, 44, 33, 24, 44, 44, 27,
        26, 28, 44, 27, 33, 44, 29, 29, 42, 31, 44, 19, 44, 19, 33,  6, 30, 33,
        31, 33, 19, 44,  3, 44, 36, 31, 44, 19, 33, 44, 30, 33, 27, 33, 44, 21,
        27, 44, 44, 33, 30, 31, 30, 33, 30, 29, 43, 29, 31, 29, 28, 16, 44, 29,
        33, 29, 44, 30, 33, 29, 40, 29, 19, 44, 43, 43, 16, 30, 19, 44, 36, 44,
        33, 33, 33, 33, 31, 14, 43, 36, 21, 36, 44, 31, 44, 36, 43,  5, 44, 24,
        33, 29, 30, 42, 44, 44, 44, 19, 33, 44, 19, 40, 29, 44, 27,  3, 44, 19,
        30, 19, 33, 19, 44, 33,  5, 33, 28, 36, 29, 44, 27, 29, 36, 30, 30, 30,
        30, 19, 29, 44, 27, 30, 30, 27, 31, 33, 19, 43, 44, 40, 26, 44, 44, 33,
        33, 29, 44, 44, 44, 30, 31, 44, 30, 36, 44, 44, 44, 31, 36, 30, 36, 29,
        31, 19, 24, 33, 28, 44, 41, 44, 30, 36, 22,  7, 33, 31, 43, 31, 27, 31,
        44, 33, 10, 33, 30, 44, 43, 30, 44, 44, 44, 15, 44, 44])

In [ ]:
# 제출
comp.submit()

[Document Classification 평가 결과]
 AI Essential 0203(1) 과정 <sol>님의 점수는 0.0660377358490566 입니다.